# Task 04 : Merge dữ liệu

# Nhiệm vụ 4: Tạo tập dữ liệu hoàn chỉnh (Case Study 03 – Quản Lý Dịch Vụ Giao Hàng)

Trong nhiệm vụ này, chúng ta sẽ thực hiện các bước sau để tạo ra bảng dữ liệu hoàn chỉnh phục vụ cho phân tích sâu hơn:

- **Đọc dữ liệu**: Tải các file đã được làm sạch gồm `shipper_info_clean.csv`, `order_info_clean.csv`, `customer_rating_clean.csv`.
- **Merge dữ liệu shipper và order**: Kết hợp thông tin shipper với thông tin đơn hàng theo `shipper_id`.
- **Merge dữ liệu order và rating**: Kết hợp thông tin đánh giá (rating) của khách hàng theo `order_id`.
- **Phát hiện mismatches**: Kiểm tra các trường hợp không khớp dữ liệu như đơn hàng không có đánh giá, shipper không tìm thấy, v.v.
- **Lưu kết quả**: Xuất bảng dữ liệu đã kết hợp đầy đủ ra file `merged_data.csv`.



In [ ]:

import pandas as pd
import numpy as np
import warnings

# Tắt cảnh báo để đầu ra gọn gàng
warnings.filterwarnings("ignore")

Đọc dữ liệu từ file CSV đã làm sạch

In [ ]:
# Đọc dữ liệu shipper, đơn hàng, và đánh giá từ file CSV đã làm sạch
shipper_info = pd.read_csv("shipper_info_clean.csv")
order_info = pd.read_csv("order_info_clean.csv", skipinitialspace=True)  # skipinitialspace để loại bỏ khoảng trắng đầu ô
customer_rating = pd.read_csv("customer_rating_clean.csv")


Sau khi đọc dữ liệu, ta kiểm tra sơ bộ kích thước (số dòng, cột) của từng bảng để đảm bảo dữ liệu đã được load đúng:

In [ ]:
print("shipper_info:", shipper_info.shape)
print("order_info:", order_info.shape)
print("customer_rating:", customer_rating.shape)
# Xem nhanh một vài dòng đầu để đảm bảo dữ liệu
print(shipper_info.head(2))
print(order_info.head(2))
print(customer_rating.head(2))


Chuẩn hóa dữ liệu khóa để merge

Trước khi kết hợp bảng, cần đảm bảo các giá trị khóa (`shipper_id`,`order_id`) không có khoảng trắng thừa và có cùng định dạng. 
- Ta loại bỏ khoảng trắng đầu/cuối của cột `shipper_id` ở cả hai bảng `shipper_info` và `order_info`.


In [ ]:
# Loại bỏ khoảng trắng trước/sau trong cột shipper_id
shipper_info['shipper_id'] = shipper_info['shipper_id'].str.strip()
order_info['shipper_id'] = order_info['shipper_id'].str.strip()

# (Tùy chọn) Có thể chuyển đổi kiểu dữ liệu nếu cần, ở đây các ID đã là string đúng định dạng.

Kiểm tra lại một số giá trị shipper_id đã xử lý:


In [ ]:
print(order_info['shipper_id'].unique()[:5])  # in ra vài shipper_id để kiểm tra


Merge thông tin shipper với order theo shipper_id

Merge bảng `order_info` với `shipper_info` theo cột `shipper_id`. 
Sử dụng `how='left'` để giữ tất cả đơn hàng, đồng thời xác định các đơn hàng mà shipper không khớp (không có thông tin shipper tương ứng).


In [ ]:
merged_order_shipper = pd.merge(order_info, shipper_info, on="shipper_id", how="left", indicator=True)
# Đổi tên cột chỉ báo kết quả merge cho dễ hiểu
merged_order_shipper.rename(columns={'_merge': 'shipper_match'}, inplace=True)
print("Kết quả merge order và shipper:", merged_order_shipper.shape)


Cột `shipper_match` trong `merged_order_shipper` chứa thông tin kết quả merge:
- `both`: `shipper_id` có trong cả `order_info` và `shipper_info`.
- `left_only`: `shipper_id` chỉ có trong `order_info` (không tìm thấy trong `shipper_info`).


In [ ]:
# Các đơn hàng không tìm thấy thông tin shipper tương ứng
no_shipper_info = merged_order_shipper[merged_order_shipper['shipper_match'] == 'left_only']
print("Số đơn hàng không tìm thấy thông tin shipper (shipper_id không khớp):", no_shipper_info.shape[0])
no_shipper_info.head(3)


Merge thêm thông tin đánh giá (rating) theo order_id

Tiến hành merge bảng kết quả trước với bảng `customer_rating` theo `order_id`. 
Dùng `how='left'` để giữ tất cả đơn hàng (cả có và không có đánh giá), đồng thời tạo cột chỉ báo `rating_match`.


In [ ]:
merged_full = pd.merge(merged_order_shipper, customer_rating, on="order_id", how="left", indicator=True)
merged_full.rename(columns={'_merge': 'rating_match'}, inplace=True)
print("Kích thước bảng hợp nhất cuối cùng:", merged_full.shape)


Trong bảng `merged_full`, cột `rating_match` có ý nghĩa:
- `both`: đơn hàng có đánh giá trong `customer_rating`.
- `left_only`: đơn hàng trong `order_info` nhưng **không** có đánh giá tương ứng.
- `right_only`: (nếu có) nghĩa là có đánh giá cho `order_id` không tồn tại trong `order_info`.


In [ ]:
# Đơn hàng không có đánh giá
no_rating = merged_full[merged_full['rating_match'] == 'left_only']
print("Số đơn hàng không có đánh giá:", no_rating.shape[0])
no_rating.head(3)


In [ ]:
# Đánh giá không khớp với đơn hàng (nếu có)
no_order_for_rating = merged_full[merged_full['rating_match'] == 'right_only']
print("Số bản ghi đánh giá không tìm thấy đơn hàng tương ứng:", no_order_for_rating.shape[0])


Phát hiện các khác biệt (mismatch) khác

Ngoài ra, có thể kiểm tra thêm:
- **Đơn hàng có nhiều hơn một đánh giá**: Một `order_id` xuất hiện nhiều lần trong `customer_rating`.
- **Shipper không có đơn hàng**: `shipper_id` trong `shipper_info` nhưng không xuất hiện trong `order_info`.


In [ ]:
# Kiểm tra đơn hàng có nhiều hơn 1 đánh giá
rating_counts = customer_rating.groupby('order_id').size().reset_index(name='count')
multiple_ratings = rating_counts[rating_counts['count'] > 1]
print("Các order_id có nhiều hơn 1 đánh giá (count > 1):")
print(multiple_ratings if not multiple_ratings.empty else "Không có order nào có nhiều hơn 1 đánh giá.")


In [ ]:
# Kiểm tra shipper không có đơn hàng
shippers_with_orders = set(order_info['shipper_id'])
unused_shippers = [s for s in shipper_info['shipper_id'] if s not in shippers_with_orders]
print("\nSố shipper_id không có đơn hàng nào:", len(unused_shippers))
print("Danh sách shipper_id không có đơn hàng:", unused_shippers)


Lưu kết quả

Cuối cùng, ta lưu bảng dữ liệu đã kết hợp (`merged_full`) ra file `merged_data.csv` để phục vụ cho các bước phân tích tiếp theo.


In [ ]:
merged_full.to_csv("merged_data.csv", index=False)
print("Đã lưu dữ liệu hợp nhất vào file 'merged_data.csv'.")


Như vậy, ta đã hoàn thành việc tạo file dữ liệu `merged_data.csv` kết hợp đầy đủ thông tin từ `shipper_info_clean`, `order_info_clean` và `customer_rating_clean`. 
Các bước kiểm tra và phát hiện mismatch đã giúp đảm bảo dữ liệu đầu ra đầy đủ và chính xác, sẵn sàng cho phân tích sâu hơn.
